# Apply document classifier

This script applies the trained classifier to scraped documents and narrows to mostly likely doi document for each district.

In [1]:
import pandas as pd
import os
import spacy
from spacy.util import minibatch, compounding
from start import data_path

/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


## Import texts

In [2]:
texts_scraped = pd.read_csv(os.path.join(data_path, 'text_scraped.csv'))
texts = texts_scraped['text']
texts.head()

0    0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...
1    Carlisle Independent School District District ...
2    Lockney ISD District of Innovation Plan House ...
3    Adopted AISD Local Innovation Plan 02-16-17 co...
4    March 2017 through March 2022 Paris ISD Distri...
Name: text, dtype: object

## Import and apply model

In [3]:
# Load model
model_dir = os.path.join(os.getcwd(),  '..', '..', 'data', 'plans', 'document_classifier')
# apply the saved model
print("Loading from", model_dir)
nlp = spacy.load(model_dir)
categories = []
for text in texts:
    doc = nlp(text)
    categories.append(doc.cats)

Loading from /Users/kylieleblancKylie/domino/dofis/code/data_from_plans/../../data/plans/document_classifier


/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/Users/kylieleblancKylie/anaconda/envs/spacy/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192, got 176
  return f(*args, **kwds)


In [4]:
p_innovation = [x['INNOVATION'] for x in categories]

In [5]:
texts_scraped_p = texts_scraped
texts_scraped_p['p_innovation'] = p_innovation
texts_scraped_p.tail()

,Unnamed: 0,Unnamed: 0.1,title,level,type,link,text,p_innovation
3990,3990,396,Petersburg ISD,html,html,http://www.petersburgisd.net/DOI,District of Innovation - Petersburg ISD PISD S...,0.109517
3991,3991,397,Three Way ISD,html,html,https://www.twisd.us/144516_2,Three Way Independent School District - Three ...,0.068096
3992,3992,398,Sinton ISD,html,html,http://www.sintonisd.net/wp/index.php/district...,District of Innovation (DOI) Latest: Teacher o...,0.012145
3993,3993,399,Athens ISD,html,html,http://athensisd.net/cms/One.aspx?portalId=841...,District of Innovation Plan - Athens Independe...,0.019236
3994,3994,400,Spring ISD,html,html,http://www.springisd.org/doi,District of Innovation / Homepage Skip to Main...,0.024606


In [6]:
texts_scraped_p.head()

,Unnamed: 0,Unnamed: 0.1,title,level,type,link,text,p_innovation
0,0,0,Abilene ISD,First,pdf,https://www.abileneisd.org/wp-content/uploads/...,0 ABILENE INDEPENDENT SCHOOL DISTRICT Local In...,0.999955
1,1,1,Carlisle ISD,First,pdf,https://s3.amazonaws.com/scschoolfiles/83/carl...,Carlisle Independent School District District ...,0.999876
2,2,2,Lockney ISD,First,pdf,https://s3.amazonaws.com/scschoolfiles/777/doi...,Lockney ISD District of Innovation Plan House ...,0.996565
3,3,3,Arlington ISD,First,pdf,http://w4.aisd.net/pdf/District-of-Innovation-...,Adopted AISD Local Innovation Plan 02-16-17 co...,0.999867
4,4,4,Paris ISD,First,pdf,https://s3.amazonaws.com/scschoolfiles/167/doi...,March 2017 through March 2022 Paris ISD Distri...,0.999955


In [7]:
texts_scraped_p.to_csv(os.path.join(data_path, 'text_scraped_p.csv'))

## Descriptive statistics

In [8]:
print("Predicted probability of DOI plan for first level documents:")
texts_scraped_p[['p_innovation']][texts_scraped_p.level == 'First'].mean()

Predicted probability of DOI plan for first level documents:


p_innovation    0.639011
dtype: float64

In [9]:
print("Predicted probability of DOI plan for second level and HTML documents:")
texts_scraped_p[['p_innovation']][texts_scraped_p.level != 'First'].mean()

Predicted probability of DOI plan for second level and HTML documents:


p_innovation    0.177814
dtype: float64

# Narrow Documents

Narrow documents to most likely DOI plan candidates. 

In [10]:
text_narrowed = texts_scraped_p.sort_values(['title', 'p_innovation'], ascending=False).drop_duplicates(subset=["title"], keep="first")

In [11]:
text_narrowed.to_csv(os.path.join(data_path, 'text_narrowed.csv'))

## Test performance

sample 30, label, what p are true doi plans?

In [12]:
retest_performance = False

In [13]:
if retest_performance:
    test = pd.read_csv(os.path.join(data_path, 'text_narrowed.csv'))
    test = test.sample(n=30)

In [14]:
# Make first 30 documents easy to read.
for i, v in enumerate(test.loc[:,:]['text'][0:30]):
    print('\n')
    print('-'*30)
    print('DISTRICT {}'.format(i))
    print('-'*30)
    print('\n')
    print(v[0:4000])

NameError: name 'test' is not defined

In [ ]:
ground_truth = [
    True, False, True, True, True,
    True, True, True, True, True,
    True, True, False, True, True,
    True, True, True, True, True,
    True, True, True, True, True,
    True, True, True, True, True
]

In [ ]:
print("% of remaining document that are DOI plans: ",sum(ground_truth) / float(len(ground_truth)))

## Narrowed Descriptives

In [ ]:
test = pd.read_csv(os.path.join(data_path, 'text_narrowed.csv'))

In [ ]:
print('Average probability of DOI classification: ', test.p_innovation.mean())
print('SD of DOI classification: ',test.p_innovation.std())
print('Number of documents w/<.2 probability of DOI classification: ', len(test[test.p_innovation < .2]))